In [1]:
%useLatestDescriptors
%use lets-plot, lets-plot-dataframe, dataframe

In [2]:
// csv from https://wahlergebnis.saarland.de/LTW/
val votesColumn by column<Double>("Anzahl")
val partyColumn by column<String>("Partei")
val df = DataFrame.readCSV("Saarlandwahlergebnis.csv", skipLines = 6).select(partyColumn, votesColumn).filter { it[partyColumn] !in listOf("Wahlberechtigte", "Wähler", "Ungültige Stimmen", "Gültige Stimmen")}
df

DataFrame: rowsCount = 18, columnsCount = 2

In [3]:
val allVotes: Double = df.sum(votesColumn)
val seatesWithElectoralThreshhold = SeatConfig("Sitze (5% Hürde)", 0.05)
val seatesWithoutElectoralThreshhold = SeatConfig("Sitze (0% Hürde)", 0.0)

val seatGoal = 51
data class SeatConfig(val columnName: String, val threshhold: Double)
val newDf = listOf(seatesWithElectoralThreshhold, seatesWithoutElectoralThreshhold).fold(df) { dataFrame, columnConfig ->
    var result = dataFrame.add(columnConfig.columnName) {0}
    var min: Int = 1
    var max = allVotes.toInt()
    var sum = result.sum(columnConfig.columnName).toInt()
    while(sum != seatGoal) {
        val quotient = (max + min) / 2
        result = result.update(columnConfig.columnName) with {
            val votes = this[votesColumn]
            // Divisorverfahren mit Abrundung
            if (votes / allVotes < columnConfig.threshhold) 0 else  (votes / quotient).toInt()
        }
        sum = result.sum(columnConfig.columnName).toInt()
        when {
            sum > seatGoal -> min = quotient
            sum < seatGoal -> max = quotient
        }
    }
    result
}
val sitze = newDf.sum(seatesWithElectoralThreshhold.columnName).toDouble()
val sitzeOhneH = newDf.sum(seatesWithoutElectoralThreshhold.columnName).toDouble()
val plot = letsPlot(newDf.toMap()) { x = partyColumn.name() }
println(newDf)
println("Sitze mit Hürde: $sitze")
println("Sitze ohne Hürde: $sitzeOhneH")

                  Partei   Anzahl Sitze (5% Hürde) Sitze (0% Hürde)
  0                  CDU 129156,0               19               16
  1                  SPD 196799,0               29               25
  2            DIE LINKE  11689,0                0                1
  3                  AfD  25718,0                3                3
  4                GRÜNE  22598,0                0                2
  5                  FDP  21618,0                0                2
  6              FAMILIE   3836,0                0                0
  7              PIRATEN   1318,0                0                0
  8         FREIE WÄHLER   7636,0                0                1
  9             dieBasis   6449,0                0                0
 10            bunt.saar   6216,0                0                0
 11                  ÖDP    615,0                0                0
 12       Die Humanisten    233,0                0                0
 13           Die PARTEI   4715,0               

In [4]:
plot + geomBar(stat = Stat.identity) { y = seatesWithElectoralThreshhold.columnName  }

In [5]:
plot + geomBar(stat = Stat.identity) { y = seatesWithoutElectoralThreshhold.columnName }